In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf


tickers = ['AAPL' , 'GOOGL', 'BRK-B' , 'PFE.DE','GC=F'] #definimos os tickers das ações 
## Ações 
prices = yf.download(tickers=tickers , start = '2017-01-01', end='2020-01-01')['Adj Close']
prices.dropna(inplace=True)

returns = (prices/prices.shift(1) -1)
returns.dropna(inplace=True)
ativos = returns.columns.to_list()
returns


[*********************100%%**********************]  6 of 6 completed


Ticker,AAPL,BRK-B,BTC-USD,GC=F,GOOGL,PFE.DE
Date,,,,,,
2017-01-04,-0.001119,0.001526,0.106233,0.002930,-0.000297,0.000474
2017-01-05,0.005085,-0.004754,-0.122410,0.013662,0.006499,-0.004895
2017-01-06,0.011148,0.000674,-0.109711,-0.006612,0.014994,0.007299
2017-01-09,0.009160,-0.008506,0.000695,0.009898,0.002387,-0.001260
2017-01-10,0.001008,-0.003395,0.005373,0.000591,-0.001414,-0.004259
...,...,...,...,...,...,...
2019-12-19,0.001000,0.000889,-0.010164,0.003803,0.003351,0.005747
2019-12-20,-0.002071,0.004884,0.002217,-0.002368,-0.003848,0.020000
2019-12-23,0.016318,-0.003668,0.018952,0.005289,-0.000437,-0.008403


In [18]:
### starting portfolio simulation
number_sim = 3000
weight = np.zeros((number_sim, len(tickers)))
Sigma = returns.cov() # matriz de covariancia
mean_returns = returns.mean()

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio  = np.zeros(number_sim)





for k in range(number_sim):
    # gerar pesos aleatórios e normalizados
    w = np.array(np.random.random(len(tickers)))
    w= w/np.sum(w)
    weight[k,:]= w 

    #retorno log esperado:
    expected_return[k] = np.sum( mean_returns *w) *252 # anualizado

    # risco: 
    #expected_risk  = w.T @ Sigma @ w
    expected_risk[k]  = np.sqrt(np.dot(w.T, np.dot(Sigma, w)) * 252) 

    sharpe_ratio[k]   = expected_return[k]/ expected_risk[k]


In [19]:
import numpy as np
import pandas as pd

# Assumindo que 'returns' é o DataFrame com os retornos diários das ações
# e que 'returns' já foi carregado anteriormente

number_sim = 1000
num_assets = len(tickers)  # Número de ativos
weight = np.zeros((number_sim, num_assets))
Sigma = returns.cov() # matriz de covariancia
mean_returns = returns.mean()

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio = np.zeros(number_sim)

for k in range(number_sim):
    # Gerar pesos aleatórios e normalizados
    w = np.random.random(num_assets)
    w = w / np.sum(w)
    weight[k, :] = w 

    # Retorno log esperado (anualizado)
    expected_return[k] = np.sum(mean_returns * w) * 252

    # Risco (anualizado)
    expected_risk[k] = np.sqrt(np.dot(w.T, np.dot(Sigma, w)) * 252)

    # Índice de Sharpe
    sharpe_ratio[k] = (expected_return[k] / expected_risk[k]) 

# Criar DataFrame
portfolio_df = pd.DataFrame(weight, columns=[f'{tickers[i]}' for i in range(num_assets)])
portfolio_df['Expected Return(%)'] = expected_return*100
portfolio_df['Risk(%)'] = expected_risk *100
portfolio_df['Sharpe Ratio'] = sharpe_ratio



portfolio_df


,AAPL,GOOGL,BRK-B,PFE.DE,GC=F,BTC-USD,Expected Return(%),Risk(%),Sharpe Ratio
0,0.153419,0.047165,0.002935,0.271365,0.074373,0.450743,14.874717,10.171401,1.462406
1,0.176262,0.000411,0.298158,0.303049,0.111049,0.111071,42.222373,25.685176,1.643842
2,0.205283,0.151751,0.055617,0.252734,0.172524,0.162091,22.295652,11.665392,1.911265
3,0.354304,0.064235,0.247064,0.146655,0.149884,0.037859,42.937120,23.792307,1.804664
4,0.109304,0.019771,0.261705,0.060575,0.246239,0.302405,38.602240,23.797826,1.622091
...,...,...,...,...,...,...,...,...,...
995,0.095777,0.245162,0.196066,0.347917,0.061361,0.053718,31.100375,18.008395,1.726993
996,0.199280,0.140222,0.312870,0.045349,0.168067,0.134211,45.107620,27.712475,1.627701
997,0.164920,0.280920,0.208438,0.255639,0.082380,0.007702,34.356591,19.667792,1.746845
998,0.182232,0.049427,0.145886,0.232488,0.222296,0.167671,30.008758,15.844872,1.893910


In [20]:
import plotly.express as px

# Encontrar a linha com o máximo índice de Sharpe
max_sharpe_idx = portfolio_df['Sharpe Ratio'].idxmax()
max_sharpe_portfolio = portfolio_df.loc[max_sharpe_idx]

# Encontrar a linha com a mínima variância (risco)
min_variance_idx = portfolio_df['Risk(%)'].idxmin()
min_variance_portfolio = portfolio_df.loc[min_variance_idx]

# Automatizar a criação do dicionário hover_data
hover_data = {col: ':.4f' for col in portfolio_df.columns}

# Criar gráfico de dispersão com Plotly, incluindo os pesos no hover_data
fig = px.scatter(
    portfolio_df, 
    x='Risk(%)', 
    y='Expected Return(%)', 
    color='Sharpe Ratio',
    title='Risco vs Retorno com Índice de Sharpe por  Monte Carlo',
    labels={'Risk(%)': 'Risco (%)', 'Expected Return(%)': 'Retorno (%)'},
    color_continuous_scale=px.colors.sequential.Bluered,
    hover_data=hover_data
)

# Adicionar contorno aos marcadores de dispersão
fig.update_traces(marker=dict(line=dict(width=2, color='black')))

# Adicionar marcadores para o portfólio com máximo Sharpe e mínima variância
fig.add_scatter(
    x=[max_sharpe_portfolio['Risk(%)']], 
    y=[max_sharpe_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='green', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Máximo Sharpe',
)

fig.add_scatter(
    x=[min_variance_portfolio['Risk(%)']], 
    y=[min_variance_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='blue', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Mínima Variância'
)

# Atualizar layout para mover a legenda para a parte inferior
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

# Mostrar gráfico
fig.show()


In [21]:
correlation_matrix = round(returns.corr(), 3)
# Mascarar a matriz de correlação para obter apenas a triangular inferior
mask = np.tril(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.mask(~mask)

# Plotar a matriz de correlação usando Plotly
fig = px.imshow(
    correlation_matrix_masked, 
    text_auto=True, 
    aspect="equal", 
    color_continuous_scale='Blues',  # Escala de cor azul
    title="Matriz de Correlação"
)

# Melhorar o aspecto do gráfico
fig.update_layout(
    width=600,  # Largura da figura
    height=600,  # Altura da figura
    margin=dict(l=100, r=50, t=50, b=50),  # Margens
    font=dict(size=14),  # Tamanho da fonte
    plot_bgcolor='white',  # Cor de fundo branca
    paper_bgcolor='white'  # Cor de fundo branca
)

fig.show()

### Vamos analisar em termos de retornos como a carteira se sairia daqui 

In [22]:
min_variance_portfolio

AAPL                   0.072331
GOOGL                  0.110281
BRK-B                  0.046235
PFE.DE                 0.543459
GC=F                   0.061403
BTC-USD                0.166290
Expected Return(%)    16.682306
Risk(%)                8.395904
Sharpe Ratio           1.986958
Name: 951, dtype: float64

In [23]:
max_sharpe_portfolio

AAPL                   0.357265
GOOGL                  0.006625
BRK-B                  0.051563
PFE.DE                 0.360169
GC=F                   0.068042
BTC-USD                0.156336
Expected Return(%)    24.490578
Risk(%)               11.792947
Sharpe Ratio           2.076714
Name: 164, dtype: float64

In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

tickers.append('SPY')

# Criar o dicionário de pesos automaticamente a partir dos tickers
weights = {ticker: max_sharpe_portfolio[ticker] for ticker in tickers if ticker in max_sharpe_portfolio.index}

# Baixar os preços ajustados das ações e do índice SPY
prices = yf.download(tickers=tickers, start='2022-01-01', end='2024-01-01')['Adj Close']
prices.dropna(inplace=True)

# Calcular os retornos diários
returns = prices.pct_change().dropna()

# Calcular o retorno diário do portfólio
portfolio_returns = returns[list(weights.keys())].mul(list(weights.values()), axis=1).sum(axis=1)

# Calcular o retorno acumulado do portfóliocumprod(
cumulative_portfolio_returns = ((1 + portfolio_returns).cumprod() - 1)*100

# Calcular o retorno acumulado do SPY
cumulative_spy_returns = ((1 + returns['SPY']).cumprod() - 1)*100

# Plotar os retornos acumulados
fig = go.Figure()

fig.add_trace(go.Scatter(x=cumulative_portfolio_returns.index, y=cumulative_portfolio_returns, 
                         mode='lines', name='Portfólio', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=cumulative_spy_returns.index, y=cumulative_spy_returns, 
                         mode='lines', name='SPY', line=dict(color='red')))

fig.update_layout(title='Retorno Acumulado do Portfólio vs SPY',
                  xaxis_title='Data',
                  yaxis_title='Retorno Acumulado(%)',
                  template='plotly_white')

fig.show()

[*********************100%%**********************]  7 of 7 completed
